## Câu 1: 

a. Dùng BLEU score.

b. Dùng W2V và sử dụng Euclid

In [17]:
%cd /content/drive/MyDrive/Full_source/NhanTruong

[Errno 2] No such file or directory: '/content/drive/MyDrive/Full_source/NhanTruong'
/content


In [18]:
# import thư viện
import os
import pandas as pd
import numpy as np
import json

root_path = '/content/drive/MyDrive/NhanTruong'

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# đường dẫn data
csv_path = os.path.join(root_path, 'dữ-liệu-chatbot-question-answer-short-style.csv')

In [21]:
# mở file csv
csv_raw_data = pd.read_csv(csv_path)

In [22]:
# Show 05 dòng đầu tiên
csv_raw_data.head()

,Unnamed: 0,user_a,user_b
0,0,Thích mẫu người nào,"Dễ thương, tóc dài, da trắng"
1,1,Có crush ai không,Có 1 bạn cùng lớp
2,2,Tại sao lại thích bạn dó,Vì đáp ứng những yêu cầu của tao
3,3,Có hay nói chuyện không,Hay nhắn tin messenger
4,4,Bạn kia có bắt chuyện trước không,Có đôi khi


In [23]:
# chuẩn bị 2 list: user_a và user_b
user_a = list(csv_raw_data['user_a'].values)
user_b = list(csv_raw_data['user_b'].values)

In [24]:
# chuẩn bị 1 hàm chuẩn hóa (xóa dấu, ký tự lạ, ...)
def remove_punc(string):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char  # space is also a character
    return no_punct.lower()

In [25]:
# chuẩn hóa dữ liệu (xóa dấu, ký tự lạ, ...)
normalized_user_a = [remove_punc(sentence) for sentence in user_a]
# normalized_user_b = [remove_punc(str(sentence)) for sentence in user_b]

In [26]:
# hàm so chuỗi BLEU4 của câu đầu vào với tập dữ liệu D.
from nltk.translate.bleu_score import sentence_bleu

def return_best_idx(input_seq, corpus):
    '''
    Đầu vào:
        input_seq: chuỗi văn bản đầu vào.
        corpus: tập dữ liệu câu hỏi D.
    Đầu ra:
        best_match_idx: vị trí câu có điểm BLEU cao nhất so với chuỗi đầu vào.
    '''
    thresholds = np.array([sentence_bleu([seq.split(' ')], input_seq.split(' ')) for seq in normalized_user_a])
    best_match_idx = np.argmax(thresholds)
    return thresholds, best_match_idx

In [ ]:
# a) Hỏi đáp dựa trên độ đo BLEU:

while 1:
    print('Nhập câu hỏi:')
    input_seq = input()
    input_seq = remove_punc(input_seq)
    thresholds, best_idx = return_best_idx(input_seq, normalized_user_a)
    answer = user_b[best_idx]
    print('Câu trả lời:', answer)

Nhập câu hỏi:


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Câu trả lời: Có 1 bạn cùng lớp
Nhập câu hỏi:
off
Câu trả lời: Dễ thương, tóc dài, da trắng
Nhập câu hỏi:


In [27]:
# Chuẩn bị pre-trained model W2V và word dict
!gdown --id 15VJpsF9idf2e3AyqU_NIPsP-uyE7Z_Eq

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=15VJpsF9idf2e3AyqU_NIPsP-uyE7Z_Eq
To: /content/Combined_PhoW2V_400d_embedding_matrix.npy
100% 3.13G/3.13G [00:16<00:00, 187MB/s]


In [12]:
!gdown --id 16Ub2aE0zGBjuCGJ9_73xK51a-yL9JY8V

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=16Ub2aE0zGBjuCGJ9_73xK51a-yL9JY8V
To: /content/word_dict.json
100% 11.3M/11.3M [00:00<00:00, 25.2MB/s]


In [28]:
# load ma trận W2V
w2v_matrix = np.load('Combined_PhoW2V_400d_embedding_matrix.npy', allow_pickle=True)

In [29]:
# load word dict
word_dict = json.load(open('word_dict.json', 'r'))
word_to_idx = {k:idx for idx, k in enumerate(word_dict)}
idx_to_word = {word_to_idx[k]:k for k in word_to_idx}

In [30]:
# Xây dựng hàm lấy ra vector biểu diễn nhúng
max_seq_len = 100 # độ dài câu tối đa
dim_vector = 400
def get_word_embedding_vector(input_seq, word_to_idx, w2v_matrix):
    '''
    Đầu vào
        input_seq: chuỗi đầu vào
        word_to_idx: từ điển chuyển từ word -> idx
        w2v_matrix: ma trận vector nhúng
    Đầu ra:
        word_embedding_vector: vecto nhúng của chuỗi đầu vào
    '''
    input_seq_split = input_seq.split(' ')
    seq_to_ids = []
    for seq in input_seq_split:
        if seq in word_to_idx:
            seq_to_ids.append(word_to_idx[seq])
        else:
            seq_to_ids.append(word_to_idx['unknown'])
    word_embedding_vector = w2v_matrix[seq_to_ids]
    pad_vector = np.zeros((max_seq_len - len(seq_to_ids), dim_vector))
    word_embedding_vector = np.concatenate((word_embedding_vector, pad_vector), 0)
    return word_embedding_vector

In [31]:
from scipy.spatial import distance
def return_best_idx_by_w2v_and_euclid_distance(input_seq, corpus):
    '''
    Đầu vào:
        input_seq: chuỗi văn bản đầu vào.
        corpus: tập dữ liệu câu hỏi D.
    Đầu ra:
        best_match_idx: vị trí câu có điểm BLEU cao nhất so với chuỗi đầu vào.
    '''
    thresholds = []
    wv_input_sentence = get_word_embedding_vector(input_seq, word_to_idx, w2v_matrix)
    for seq in corpus:
        wv_seq = get_word_embedding_vector(seq, word_to_idx, w2v_matrix)
        score = distance.euclidean(wv_input_sentence.reshape(-1,1), wv_seq.reshape(-1,1))
        thresholds.append(score)
        
    best_match_idx = np.argmin(thresholds)
    return thresholds, best_match_idx

In [ ]:
while 1:
    print('Nhập câu hỏi:')
    input_seq = input()
    input_seq = remove_punc(input_seq)
    thresholds, best_idx = return_best_idx_by_w2v_and_euclid_distance(input_seq, normalized_user_a)
    answer = user_b[best_idx]
    print('Câu trả lời:', answer)

Nhập câu hỏi:
Tên gì
Câu trả lời: Lan Anh.
Nhập câu hỏi:
